
# Modelo de red neuronal mejorado para detenidos 2024

Este notebook mejora el modelo anterior utilizando técnicas adicionales de **preprocesamiento** y **optimización**. Se usa `StandardScaler` para estandarizar la variable numérica `edad` (centrar la media en 0 y escalar a varianza unitaria) porque muchos algoritmos de aprendizaje pueden comportarse mal si las características tienen escalas muy diferentes【641495248861329†L690-L703】. También se aplica `OneHotEncoder` para convertir las variables categóricas en columnas binarias, lo que permite que la red neuronal procese datos categóricos【415078688353375†L679-L691】.  A continuación, se entrena un perceptrón multicapa (`MLPClassifier`) con dos capas ocultas y parada temprana (`early_stopping=True`) para mejorar la generalización.



In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score


In [ ]:

# Cargar datos
df = pd.read_csv('mdi_detenidos_2024_limpio.csv')

# Eliminar columna de alta cardinalidad
if 'codigo_provincia' in df.columns:
    df = df.drop(columns=['codigo_provincia'])

# Convertir edad a numérico y eliminar filas con valores faltantes
df['edad'] = pd.to_numeric(df['edad'], errors='coerce')
df = df.dropna(subset=['edad'])

# Rellenar valores faltantes en variables categóricas
df = df.fillna('Desconocido')

# Simplificar presunta_infraccion a las 3 categorías más frecuentes + OTROS
top3 = df['presunta_infraccion'].value_counts().nlargest(3).index
df['presunta_infraccion'] = df['presunta_infraccion'].apply(lambda x: x if x in top3 else 'OTROS')

# Muestra de 50% para acelerar el entrenamiento
if len(df) > 20000:
    df = df.sample(frac=0.5, random_state=42)

# Codificar objetivos
y_tipo = LabelEncoder().fit_transform(df['tipo'])
y_infr = LabelEncoder().fit_transform(df['presunta_infraccion'])

# Definir variables predictoras (excluyendo objetivos)
X = df.drop(columns=['tipo', 'presunta_infraccion'])

# Identificar columnas numéricas y categóricas
numeric_features = ['edad']
categorical_features = [col for col in X.columns if col not in numeric_features]

# Preprocesador: escalar y codificar
preprocess = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# División entrenamiento/prueba para ambos objetivos
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X, y_tipo, test_size=0.2, random_state=42, stratify=y_tipo)
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X, y_infr, test_size=0.2, random_state=42, stratify=y_infr)


In [ ]:

# Modelo para 'tipo' (clasificación binaria)
tipo_model = Pipeline([
    ('preprocess', preprocess),
    ('mlp', MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=200, random_state=42, early_stopping=True))
])

# Entrenar y evaluar
print('Entrenando modelo para tipo...')
tipo_model.fit(X_train_t, y_train_t)
y_pred_t = tipo_model.predict(X_test_t)

print("
=== Reporte para 'tipo' mejorado ===")
print(classification_report(y_test_t, y_pred_t, target_names=['APREHENDIDO','DETENIDO']))
print('Precisión de tipo:', accuracy_score(y_test_t, y_pred_t))


In [ ]:

# Modelo para 'presunta_infraccion' (multiclase)
infr_model = Pipeline([
    ('preprocess', preprocess),
    ('mlp', MLPClassifier(hidden_layer_sizes=(150, 75), activation='relu', solver='adam', max_iter=200, random_state=42, early_stopping=True))
])

# Entrenar y evaluar
print('Entrenando modelo para presunta_infraccion...')
infr_model.fit(X_train_i, y_train_i)
y_pred_i = infr_model.predict(X_test_i)

# Obtener nombres de clases
target_names_infr = LabelEncoder().fit(df['presunta_infraccion']).classes_
print("
=== Reporte para 'presunta_infraccion' mejorado ===")
print(classification_report(y_test_i, y_pred_i, target_names=target_names_infr))
print('Precisión de presunta_infraccion:', accuracy_score(y_test_i, y_pred_i))
